In [2]:
import pandas as pd
import os
from itertools import chain
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
directory = '../Data'  # Directory where the pickle files are stored
pickle_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pickle')]

dataframes = []
for file in pickle_files:
    s = pd.read_pickle(file)
    flattened_list = list(chain.from_iterable(s))

    # Create a DataFrame
    df = pd.DataFrame(flattened_list, columns=['ID', 'DateTime'])
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)


In [4]:
counts=combined_df.groupby('ID').count().sort_values(by='ID').reset_index()
total_counts = counts['DateTime'].sum()


# Calculate the density for each 'ID'
density = counts['DateTime'] / total_counts


counts['density'] = density
grid_30x30 = np.zeros((30, 30))

# Iterate over the DataFrame and assign densities to the grid
for index, row in counts.iterrows():
    # Ensure the ID is within the valid range (0 to 899)
    if 0 <= row['ID'] < 900:
        row_idx = row['ID'] // 30  # Calculate row index
        col_idx = row['ID'] % 30   # Calculate column index
        if grid_30x30[int(row_idx), int(col_idx)]==0:
            grid_30x30[int(row_idx), int(col_idx)] = row['density']
        else:
            print("Duplicate")
            print(int(row_idx), int(col_idx))

In [5]:
def update_density_grid(current_time,combined_df):
    # Code to update the density grid every 10 days
    subset= combined_df[combined_df.DateTime<current_time+pd.Timedelta(days=10)]
    
    counts=subset.groupby('ID').count().sort_values(by='ID').reset_index()
    total_counts = counts['DateTime'].sum()
    

    # Calculate the density for each 'ID'
    density = counts['DateTime'] / total_counts


    counts['density'] = density
    counts
    grid_30x30 = np.zeros((30, 30))

    # Iterate over the DataFrame and assign densities to the grid
    for index, row in counts.iterrows():
        # Ensure the ID is within the valid range (0 to 899)
        if 0 <= row['ID'] < 900:
            row_idx = row['ID'] // 30  # Calculate row index
            col_idx = row['ID'] % 30   # Calculate column index
            if grid_30x30[int(row_idx), int(col_idx)]==0:
                grid_30x30[int(row_idx), int(col_idx)] = row['density']
            else:
                print("Duplicate")
                print(int(row_idx), int(col_idx))
    
    return grid_30x30

In [23]:
def aggregate_grid(grid, block_size):
    # Aggregate the grid into larger blocks
    num_rows, num_cols = grid.shape
    aggregated_grid = np.zeros((num_rows // block_size, num_cols // block_size))

    for i in range(0, num_rows, block_size):
        for j in range(0, num_cols, block_size):
            aggregated_grid[i // block_size, j // block_size] = np.sum(grid[i:i+block_size, j:j+block_size])
    
    return aggregated_grid

def solve_p_median(grid, p, potential_facilities):
    # Create a model
    m = gp.Model("p_median")
    num_rows, num_cols = grid.shape

    # Decision Variables
    x = m.addVars(num_rows, num_cols, vtype=GRB.BINARY, name="x")

    # Objective Function: Minimize the weighted sum of distances
    m.setObjective(gp.quicksum(grid[i, j] * (abs(i-k) + abs(j-l)) * x[k, l]
                               for i in range(num_rows) for j in range(num_cols)
                               for k, l in potential_facilities), GRB.MINIMIZE)

    # Constraints

    m.addConstr(gp.quicksum(x[k, l] for k, l in potential_facilities) == p, "facility_count")

    # Solve the model
    m.optimize()

    solution = []
    # Extract and print the solution
    if m.status == GRB.OPTIMAL:
        solution_x = m.getAttr('x', x)
        print("Facility Locations:")
        for k, l in potential_facilities:
            if solution_x[k, l] == 1:
                print(f"Facility at cell ({k}, {l})")
                solution.append([k, l])
    else:
        print("No optimal solution found")
    
    return solution
# Example Usage
original_grid = grid_30x30 
aggregated_grid = aggregate_grid(original_grid, block_size=1)

potential_facilities = [(i, j) for i in range(aggregated_grid.shape[0]) for j in range(aggregated_grid.shape[1]) if (i % 1 == 0 and j % 1 == 0)]
solution = solve_p_median(aggregated_grid, p=3, potential_facilities=potential_facilities)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 900 columns and 900 nonzeros
Model fingerprint: 0xa1225f1b
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 82.5097557
Presolve removed 1 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 15.6119 82.5098 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.561191349064e+01, best bound 1.561191349064e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 15)
Facility at cell (13, 15)
Facility at ce

In [18]:
# Main loop
current_time=combined_df.DateTime.min()
while (current_time<combined_df.DateTime.max()):
    density_grid = update_density_grid(current_time,combined_df)
    solution = solve_p_median(density_grid, 3, potential_facilities)
    current_time+=pd.Timedelta(days=10)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 900 columns and 900 nonzeros
Model fingerprint: 0x8b3d494d
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 84.4328140
Presolve removed 1 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 22.3969 84.4328 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.239691003256e+01, best bound 2.239691003256e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 16)
Facility at cell (12, 17)
Facility at ce

In [29]:
import numpy as np
def translate_to_original_grid(solution, block_size):
    original_coordinates = []
    for k, l in solution:
        centroid_row = block_size * k + block_size // 2
        centroid_col = block_size * l + block_size // 2
        original_coordinates.append((centroid_row, centroid_col))
    return original_coordinates
original_grid = grid_30x30 

# Parameters to iterate over
aggregate_grid_sizes = [1,3,5,10]
ps = [3, 4, 5, 6, 7, 8, 9]

solutions_by_p = {p: [] for p in ps}

# Iterate over each combination of aggregate grid size and number of facilities
for block_size in aggregate_grid_sizes:
    aggregated_grid = aggregate_grid(original_grid, block_size=block_size)

    for p in ps:
        print(f"\nRunning for block size {block_size} and {p} facilities.")
        potential_facilities = [(i, j) for i in range(aggregated_grid.shape[0]) for j in range(aggregated_grid.shape[1])]
        solution = solve_p_median(aggregated_grid, p, potential_facilities)
        if solution is not None:
            original_coordinates = translate_to_original_grid(solution, block_size)
            print("Optimal facility locations in the original grid:", original_coordinates)
            solutions_by_p[p].append((block_size, original_coordinates))

# Print or process the stored solutions
for p, solutions in solutions_by_p.items():
    print(f"\nSolutions for p = {p}:")
    for block_size, coordinates in solutions:
        print(f"Block size {block_size}: {coordinates}")

        



Running for block size 1 and 3 facilities.
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 900 columns and 900 nonzeros
Model fingerprint: 0xa1225f1b
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 82.5097557
Presolve removed 1 rows and 900 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 15.6119 82.5098 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.561191349064e+01, best bound 1.561191349064e+01, gap 0.0000%
Facility Locations:
Facility at cell (12, 


Optimal solution found (tolerance 1.00e-04)
Best objective 4.949517014000e+01, best bound 4.949517014000e+01, gap 0.0000%
Facility Locations:
Facility at cell (11, 15)
Facility at cell (12, 15)
Facility at cell (12, 16)
Facility at cell (13, 14)
Facility at cell (13, 15)
Facility at cell (13, 16)
Facility at cell (13, 17)
Facility at cell (14, 15)
Facility at cell (14, 16)
Optimal facility locations in the original grid: [(11, 15), (12, 15), (12, 16), (13, 14), (13, 15), (13, 16), (13, 17), (14, 15), (14, 16)]

Running for block size 3 and 3 facilities.
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 100 columns and 100 nonzeros
Model fingerprint: 0x77c36542
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1

  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective 57.5427897
Presolve removed 1 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 20.9064 57.5428 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.090638486332e+01, best bound 2.090638486332e+01, gap 0.0000%
Facility Locations:
Facility at cell (2, 5)
Facility at cell (3, 4)
Facility at cell (3, 5)
Facility at cell (3, 6)
Facility at cell (4, 4)
Facility at cell (4, 5)
Facility at cell (4, 6)
Facility at cell (5, 5)
Facility at cell (5, 6)
Optimal facility locations in the original grid: [(7, 16), (10, 13), (10, 16), (10, 19), (13, 13), (13, 16), (13, 19), (16, 16), (16, 19)]

Running for block size 5 and 3 facilities.
Gurobi Optimizer 


CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 36 columns and 36 nonzeros
Model fingerprint: 0x83df2a26
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-01, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective 31.1244003
Presolve removed 1 rows and 36 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 15.9214 31.1244 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.592139851881e+01, best bound 1.592139851881e+01, gap 0.0000%
Facility Locations:
Facility at cell (1, 2)
Facility at cell (1, 3)
Facility at cell (1, 4)
Facility at cell (2, 2)
Facility at cell (2, 3)
Facility at cell (2, 4)
Facility at


CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 9 columns and 9 nonzeros
Model fingerprint: 0xf50584b9
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective 13.3755874
Presolve removed 1 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 13.3756 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.337558743203e+01, best bound 1.337558743203e+01, gap 0.0000%
Facility Locations:
Facility at cell (0, 0)
Facility at cell (0, 1)
Facility at cell (0, 2)
Facility at cell (1, 0)
Facility at cell (1, 1)
Facility at cell (1, 2)
Facility at cell (2, 0)
